In [30]:
import polars as pl

# E-commerce Business Transaction
https://www.kaggle.com/datasets/gabrielramos87/an-online-shop-business/data


In [31]:
raw_data = pl.read_csv('./Sales_Transaction_v4a.csv', infer_schema = False)
raw_data.glimpse()

Rows: 536350
Columns: 8
$ TransactionNo <str> '581482', '581475', '581475', '581475', '581475', '581475', '581475', '581475', '581475', '581475'
$ Date          <str> '12/9/2019', '12/9/2019', '12/9/2019', '12/9/2019', '12/9/2019', '12/9/2019', '12/9/2019', '12/9/2019', '12/9/2019', '12/9/2019'
$ ProductNo     <str> '22485', '22596', '23235', '23272', '23239', '21705', '22118', '22119', '22217', '22216'
$ ProductName   <str> 'Set Of 2 Wooden Market Crates', 'Christmas Star Wish List Chalkboard', 'Storage Tin Vintage Leaf', 'Tree T-Light Holder Willie Winkie', 'Set Of 4 Knick Knack Tins Poppies', 'Bag 500g Swirly Marbles', 'Joy Wooden Block Letters', 'Peace Wooden Block Letters', 'T-Light Holder Hanging Lace', 'T-Light Holder White Lace'
$ Price         <str> '21.47', '10.65', '11.53', '10.65', '11.94', '10.65', '11.53', '12.25', '10.65', '10.55'
$ Quantity      <str> '12', '36', '12', '12', '6', '24', '18', '12', '12', '24'
$ CustomerNo    <str> '17490', '13069', '13069', '13069', '130

## Data Transformation

In [ ]:
main_data = (
    raw_data
    .select(
        pl.col('TransactionNo').str.strip_chars().alias('txn_id'),
        pl.col('Date').str.to_date(format = "%m/%d/%Y").alias('txndate'),
        pl.col('ProductNo').str.strip_chars().alias('pid'),
        pl.col('ProductName').str.strip_chars().alias('pname'),
        pl.col('Price').cast(pl.Float64).alias('price'),
        pl.col('Quantity').cast(pl.Int64).alias('quantity'),
        pl.col('CustomerNo').str.strip_chars().alias('cust_id'),
        pl.col('Country').str.strip_chars().alias('country')
    )
    .with_columns(
        payment = pl.col('price') * pl.col('quantity')
    )
)

## Data Aggregation and Cleansing

In [ ]:
# remove the cust_id == "NA"
main_data.group_by('cust_id').len().sort(by = 'cust_id')

main_data= (
    main_data
    .filter(pl.col('cust_id') != "NA")
)

In [34]:
# remove the order which has total_amount less than 0
main_data = (
    main_data
    .with_columns(
        sum_payment = pl.col('payment').sum().over(partition_by = ['txn_id', 'cust_id', 'txndate'])
    )
    .filter(pl.col('sum_payment') > 0)
    .drop('sum_payment')
)

In [35]:
# aggregation by member, date
agg_daily = (
    main_data
    .group_by('cust_id', 'txndate')
    .agg(
        # sum_price = (pl.col('price') * pl.col('quantity')).sum(),
        sum_pay = pl.col('payment').sum(),
        sum_cart = pl.col('txn_id').n_unique()
    )
    .sort(by = ['cust_id', 'txndate'])
    .with_columns(
        # sum_discnt = pl.col('sum_price') - pl.col('sum_pay'),
        gap = (pl.col('txndate') - pl.col('txndate').shift(n = 1).over(partition_by = ['cust_id'], order_by = ['txndate'])).dt.total_days(),
        rev_gap = (pl.col('txndate').shift(n = -1).over(partition_by = ['cust_id'], order_by = ['txndate']) - pl.col('txndate')).dt.total_days(),
        seq = pl.col('txndate').rank(descending = False).over(partition_by = 'cust_id').cast(pl.Int64),
        rev_seq = pl.col('txndate').rank(descending = True).over(partition_by = 'cust_id').cast(pl.Int64)
    )
)

In [36]:
# aggregation by member
agg_uid = (
    agg_daily
    .group_by('cust_id')
    .agg(
        min_date = pl.col('txndate').min(),
        max_date = pl.col('txndate').max(),
        n_days = pl.col('txndate').len(),
        sum_pay = pl.col('sum_pay').sum(),
        sum_cart = pl.col('sum_cart').sum(),
    )
    .with_columns(
        rep_count = pl.col('n_days') - 1,
        gap_mean = ((pl.col('max_date') - pl.col('min_date')).dt.total_days()) / (pl.col('n_days') - 1),
    )
    .with_columns(pl.col('gap_mean').fill_nan(value = pl.lit(None)))
)

## Export

In [37]:
main_data.write_csv("./Sales_Transaction_v4a/main.csv")
agg_daily.write_csv("./Sales_Transaction_v4a/agg_daily.csv")
agg_uid.write_csv("./Sales_Transaction_v4a/agg_uid.csv")

---

# Another Dataset Name